In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
from Deepfunction import *

In [ ]:
class Model:
    """Model Declaration"""
    
    def __init__(self):
        self.LayerList = []
        
    def add(self, layer):
        self.LayerList.append(layer)
    
    def fit(self, X_train, y_train):
        for i in range(epochs):
            for x, y in zip(X_train, y_train):
                for layer in self.LayerList:
                    x = layer.forward(x)

                for layer in self.LayerList[::-1]:
                    y = layer.backprop(y)

                for layer in self.LayerList:
                    layer.update(alpha)
    
    def score(X_test, y_test):
        score = []

        for x, y in zip(X_train, y_train):
            for layer in self.LayerList:
                x = layer.forward(x)

            score.append(np.argmax(x) == np.argmax(y))

        return np.mean(score)
            
class Layer:
    """layer"""
    
    def __init__ (self, pre_neurons, neurons, activation = 'sigmoid', num = 0, layer = 'hidden'):
        """Initializing"""
        self.W = np.random.randn(neurons, pre_neurons) * np.sqrt(1. / neurons)
        self.b = np.random.randn(neurons) * np.sqrt(1. / neurons)
        self.activation = activation
        self.num = num
        self.layer = layer
        
    def forward(self, a_in):
        """Forward"""
        self.a_in = a_in
        self.z = self.W @ self.a_in + self.b
        
        if self.activation == 'sigmoid':
            self.a_out = sigmoid(self.z)
        
        elif self.activation == 'relu':
            self.a_out = relu(self.z)
        
        return self.a_out
    
    def backprop(self, grad_a):
        """Back propagation"""
        if self.layer == 'output':
            grad_a = (self.a_out - grad_a) / len(self.a_out)
        else:
            grad_a = grad_a
            
        self.grad_b = grad_a * d_sigmoid(self.z)
        self.grad_W = np.outer(self.grad_b, self.a_in)
        self.grad_a = self.W.T @ self.grad_b

        return self.grad_a
    
    def update(self, alpha):
        self.W = self.W - alpha * self.grad_W / 250
        self.b = self.b - alpha * self.grad_b / 250

    def __str__(self):
        """Print weight and bias"""
        return (f"{self.num}'s Layer W\n{self.W}\n\n{self.num}'s Layer b\n{self.b}")
    
def score(X_test, y_test):
    score = []
    
    for x, y in zip(X_test, y_test):
        a1 = l1.forward(x)
        a2 = l2.forward(a1)
        a3 = l3.forward(a2)
        
        score.append(np.argmax(a3) == np.argmax(y))
        
    return np.mean(score)

In [ ]:
np.random.seed(45)

dataset = pd.read_csv('./MNIST_preprocessed.csv', sep=',', header=None).values


X = dataset[:, 0:784]
Y = dataset[:, 784:]
X_train, X_test = X[0:250,], X[250:,]
y_train, y_test = Y[0:250,], Y[250:,]


l1 = Layer(784,128,num=1)
l2 = Layer(128,64,num=2)
l3 = Layer(64,10,num=3,layer='output')

epochs = 30
alpha = 300

for i in range(epochs):
    for x, y in zip(X_train, y_train):
        a1 = l1.forward(x)
        a2 = l2.forward(a1)
        a3 = l3.forward(a2)
        
        b3 = l3.backprop(y)
        b2 = l2.backprop(b3)
        b1 = l1.backprop(b2)
        
        l1.update(alpha)
        l2.update(alpha)
        l3.update(alpha)
        #print(a3)
        
    print(score(X_test, y_test))